In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
from openai import OpenAI

import os

os.environ["OPENAI_API_KEY"] = "..." # Add your OpenAI key

Define model:

In [ ]:
client = OpenAI()

Define file name:

In [ ]:
fileNames = ["Verifiable", "Soft", "Non-verifiable"]

Load data:

In [ ]:
def openSegments(fileName):
  # Open collection of segments:
  f = open("drive/MyDrive/data/"+fileName+"_segments.txt", "r")

  data = []
  segmentText = ""
  label = "False"
  for line in f:
      try:
          if '>>>' in line:
              # memorize previous segment:
              if segmentText != "":
                  data.append((segmentText.strip(),label))
                  segmentText = ""
              # determine label of current segment:
              label = line.split('>>>')[1].strip()
          else:
              segmentText += line

      except ValueError:
          print('Invalid input:',line)

  # Memorize the last segment:
  data.append((segmentText.strip(),label))

  f.close()

  return data

Generate summaries and save the results:

In [ ]:
n = 3 # number of times to repeat the experiment

for i in range(n):
  for f_name in fileNames:
    # Load data:
    data = openSegments(f_name)

    # Define output file:
    f_out = open("drive/MyDrive/Summaries/"+f_name+"_summaries_"+str(i+1)+".txt","w")

    # Generate summaries for each segment:
    for segment in data:
      request = 'résumé abstrait à 20 mots maximum : "'+segment[0]+'"'
      chat_completion = client.chat.completions.create(
          messages=[
              {
                  "role": "user",
                  "content": request,
              }
          ],
          model="gpt-3.5-turbo-0301",
      )
      summary = chat_completion.choices[0].message.content

      # Output original segment:
      f_out.write('>>> '+segment[1]+'\n\n')
      f_out.write(segment[0]+'\n\n')
      f_out.write('>> Résumé :\n\n')

      # Save result to the file:
      f_out.write('> '+summary+'\n\n\n')

    f_out.close()